# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,22.82,73,17,5.04,PN,1712461804
1,1,cukai,4.2500,103.4167,30.97,60,48,2.74,MY,1712461804
2,2,edinburgh of the seven seas,-37.0676,-12.3116,18.80,95,99,9.75,SH,1712461804
3,3,kavaratti,10.5669,72.6420,29.07,70,98,1.27,IN,1712461804
4,4,steinbach,50.1667,8.5667,16.47,69,99,3.84,DE,1712461804


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City",
    alpha = 0.5
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = pd.DataFrame(city_data_df.loc[(city_data_df["Max Temp"]>17) &
                                             (city_data_df["Max Temp"]<32) &
                                             (city_data_df["Humidity"]>30) &
                                             (city_data_df["Humidity"]<80) &
                                             (city_data_df["Cloudiness"]==0)
])

# Drop any rows with null values
city_data_df.dropna(inplace=True)

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,gulf shores,30.2460,-87.7008,17.78,78,0,0.00,US,1712461806
119,119,mar del plata,-38.0023,-57.5575,19.01,69,0,5.66,AR,1712461205
162,162,ha'il,27.5219,41.6907,19.01,52,0,2.06,SA,1712461815
176,176,saipan,15.1355,145.7010,31.39,58,0,11.32,MP,1712461816
189,189,valparaiso,-33.0393,-71.6273,18.33,56,0,0.00,CL,1712461717
205,205,beroroha,-21.6667,45.1667,23.25,58,0,0.54,MG,1712461819
214,214,celestun,20.8667,-90.4000,25.00,66,0,6.31,MX,1712461820
228,228,hanover,52.3705,9.7332,20.06,58,0,4.63,DE,1712461821
233,233,carnarvon,-24.8667,113.6333,26.71,55,0,7.97,AU,1712461822
290,290,boa vista,2.8197,-60.6733,27.99,65,0,3.09,BR,1712461707


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
27,gulf shores,US,30.2460,-87.7008,78,
119,mar del plata,AR,-38.0023,-57.5575,69,
162,ha'il,SA,27.5219,41.6907,52,
176,saipan,MP,15.1355,145.7010,58,
189,valparaiso,CL,-33.0393,-71.6273,56,
205,beroroha,MG,-21.6667,45.1667,58,
214,celestun,MX,20.8667,-90.4000,66,
228,hanover,DE,52.3705,9.7332,58,
233,carnarvon,AU,-24.8667,113.6333,55,
290,boa vista,BR,2.8197,-60.6733,65,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"apiKey": geoapify_key,
          "categories": "accommodation.hotel",
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lng = hotel_df.loc[index,"Lng"]
    Lat = hotel_df.loc[index,"Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
gulf shores - nearest hotel: Beachside Resort
mar del plata - nearest hotel: Gran hotel Mónaco
ha'il - nearest hotel: فندق كلاسيك
saipan - nearest hotel: Chalan Kanoa Beach Hotel
valparaiso - nearest hotel: Ibis Hotel
beroroha - nearest hotel: No hotel found
celestun - nearest hotel: Hotel Flamingo Playa
hanover - nearest hotel: Am Leineschloss
carnarvon - nearest hotel: No hotel found
boa vista - nearest hotel: Aipana Plaza Hotel
arar - nearest hotel: دار أوتيل Dar Otel
tazacorte - nearest hotel: App Leyma


,City,Country,Lat,Lng,Humidity,Hotel Name
27,gulf shores,US,30.2460,-87.7008,78,Beachside Resort
119,mar del plata,AR,-38.0023,-57.5575,69,Gran hotel Mónaco
162,ha'il,SA,27.5219,41.6907,52,فندق كلاسيك
176,saipan,MP,15.1355,145.7010,58,Chalan Kanoa Beach Hotel
189,valparaiso,CL,-33.0393,-71.6273,56,Ibis Hotel
205,beroroha,MG,-21.6667,45.1667,58,No hotel found
214,celestun,MX,20.8667,-90.4000,66,Hotel Flamingo Playa
228,hanover,DE,52.3705,9.7332,58,Am Leineschloss
233,carnarvon,AU,-24.8667,113.6333,55,No hotel found
290,boa vista,BR,2.8197,-60.6733,65,Aipana Plaza Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 500,
    size = "Humidity",
    scale = 2,
    color = "City",
    alpha = 0.5,
    hover_cols=["Country","Hotel Name"]
)

# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)